# Conclude Session of Recommondation System

Progress so far:
1. Data Cleaning and Modification
2. Clustering on whole data
3. Data Segmentation based on consistency
4. Deep learning mode on consistent groups

In this page, I'm going to construct a re-cap of my previous work with a clean and clear version. Then made some modification on clustering and data segmentation for the sake of next step's preparation.

The idea includes:
1. Conclude M18 and 19
2. Modify the way of data segmentation (v1-v3)
3. Other algorithms of clustering
4. Confirm how the recommendation system perform
5. Make sure the system is good for new data inputs

## Data Loading and Cleaning

In [6]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
sys.path.append(r'C:\Frank\UoT 2024-2025\MIE8888 Project\M18\RUIWU') 
from helper_functions import * # Import Ruiwu's helper functions for data cleaning, slicing and aggregating

In [7]:
# Get current ful data
full_dataset = r"C:\Frank\UoT 2024-2025\MIE8888 Project\full_dataset_2.11.csv"
df = pd.read_csv(full_dataset, encoding="Windows-1252")

# Define trash columns
trash_columns = ['id_x_x','sikka_sub_task_operation_x_x','office_x_x','guarantor_x','patient','description','visible_to_patient','primary_insurance_estimate','secondary_insurance_estimate','plan_sr_no','insurance_payment','approved','from_pms_x_x','id_y_x','from_pms_y_x','sikka_sub_task_y_x','sikka_sub_task_operation_y_x','uuid','guarantor_y','office_y_x','fullname','first_name','last_name','address','phone','zip_code','practice','primary_insurance_company_id','profile_picture','date_joined','zip_code_flag','address_comp','final_zip_code','id_x_y','from_pms_x_y','office_x_y','treatment','id_y_y','from_pms_y_y','sikka_sub_task_y_y','sikka_sub_task_operation_y_y','office_y_y','procedure_code_x','procedure_code_description','patient_friendly_description','treatment_id','price','fetched_zip_code_flag','zip_code_verified','fetched_zip_code', 'entry_date', 'tooth_from']

# Data Engineering
df = drop_high_nan_columns(df) 
df = df.drop(columns = trash_columns) # Drop trash cols
df = merge_columns_with_priority(df,'health_category','procedure_code_category','treatment_category') # merge to get treatment_category with pattern from two category groups
df = df.dropna() # Drop the remaining NA
df, df_dict = object_processing(df) # Project columns dtype = object; For columns containing datetime, convert it to pd.datetime; For other object columns, encode them
df = days_cal(df,'procedure_date','birth_date', 'procedure_age') # age group preparation (modify to int)
df = days_cal(df,'last_visit','first_visit','visit_period')

# Perform one-hot encoding on the "procedure_code_y" and "treatment_category"
df = one_hot_encode_procedure_and_treatment(df)
# df.to_csv(r'C:\Frank\UoT 2024-2025\MIE8888 Project\M18\output.csv', index=False)
df_next = df.copy() # for next step use

C:\Users\frank\AppData\Local\Temp\ipykernel_9708\1752406196.py:3: DtypeWarning: Columns (1,4,10,16,52,85,86,91) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(full_dataset, encoding="Windows-1252")
c:\Frank\UoT 2024-2025\MIE8888 Project\SmilePass-2\helper_functions.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  temp_dt = pd.to_datetime(df[col], errors='raise')
c:\Frank\UoT 2024-2025\MIE8888 Project\SmilePass-2\helper_functions.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  temp_dt = pd.to_datetime(df[col], errors='raise')
c:\Frank\UoT 2024-2025\MIE8888 Project\SmilePass-2\helper_functions.py:40: UserWarning: Could not infer format, so each element will b

- Create a complete dataset to reflect each patient_id's treatment pattern (aassuming all meta columns stand the same for each unique patient id)
- Applied Truncated SVD which is the best for sparse binary data of large sparse matrices.

Note: `final_df` is generated from whole dataset; `df_svd` is based on it. Replace it with any new version of dataset if needed

In [ ]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

# Define Constant columns
meta_cols = ['provider_x', 'amount', 'treatment_plan_status', 'patient_id', 'city','state_y', 'sikka_provider_id', 'LATITUDE', 'LONGITUDE', 'procedure_age', 'visit_period']

# Select relevant columns
base_cols = ['patient_id', 'procedure_age']
procedure_cols = [col for col in df_next.columns if col.startswith('procedure_code_y_') or col.startswith('treatment_category_')]
df_subset = df_next[base_cols + procedure_cols].copy()
df_subset['procedure_age'] = df_subset['procedure_age'].astype(int) # procedure_age to int

# Melt one-hot to long format
df_long = df_subset.melt(
    id_vars=['patient_id', 'procedure_age'],
    value_vars=procedure_cols,
    var_name='procedure_code',
    value_name='occurred'
)

# Keep only rows where procedure occurred
df_long = df_long[df_long['occurred'] == 1]

# Create combined feature name
df_long['feature'] = df_long['procedure_code'] + '_age' + df_long['procedure_age'].astype(str)

# Pivot to wide format
final_df = df_long.groupby(['patient_id', 'feature']).size().unstack(fill_value=0)
final_df = final_df.reset_index()

# Get metadata (take first row per patient) # may need further fixing if the information for such patient changed over times
patient_meta = df_next[meta_cols].groupby('patient_id').first().reset_index()
final_df = pd.merge(patient_meta, final_df, on='patient_id', how='left')

# columns to exclude from SVD
non_binary_cols = ['provider_x', 'amount', 'treatment_plan_status', 'patient_id', 'city','state_y', 'sikka_provider_id', 'LATITUDE', 'LONGITUDE','procedure_age', 'visit_period']
meta_df = final_df[non_binary_cols].copy()
binary_df = final_df.drop(columns=non_binary_cols)

# Apply TruncatedSVD on binary columns
X_sparse = csr_matrix(binary_df.values)
svd = TruncatedSVD(n_components=100, random_state=823)
X_svd = svd.fit_transform(X_sparse)
svd_features = pd.DataFrame(X_svd, columns=[f"SVD_{i+1}" for i in range(X_svd.shape[1])])
df_svd = pd.concat([meta_df.reset_index(drop=True), svd_features], axis=1)

- Dataframe for 17 age groups, start from 5-10, up to 85-90
- Dataframe for each age groups

Note: Change `df` to any version of `dataset` if needed

In [ ]:
import os

# 5 year window
def age_to_group(age):
    try:
        age = int(age)
        if age < 0:
            return "invalid"
        lower = (age // 5) * 5
        upper = lower + 5
        if lower > upper:
            return "invalid"
        return f"{lower}-{upper}"
    except:
        return "invalid"

df['age_group'] = df['procedure_age'].apply(age_to_group)
output_dir = r"C:\Frank\UoT 2024-2025\MIE8888 Project\M18\Split dataframe 5 year"
os.makedirs(output_dir, exist_ok=True)
# Split and save each group
for group_name, group_df in df.groupby('age_group'):
    filename = f"{output_dir}/age_group_{group_name}.csv"
    group_df.to_csv(filename, index=False)
    print(f"Saved: {filename}")


# 1 year window
def age_to_group(age):
    try:
        age = int(age)
        if age < 0:
            return "invalid"
        return f"{age}-{age + 1}"
    except:
        return "invalid"

df['age_group'] = df['procedure_age'].apply(age_to_group)
output_dir = r"C:\Frank\UoT 2024-2025\MIE8888 Project\M18\Split dataframe yearly"
os.makedirs(output_dir, exist_ok=True)
# Split and save each group
for group_name, group_df in df.groupby('age_group'):
    if group_name == "invalid":
        continue  # Skip invalid entries
    filename = os.path.join(output_dir, f"age_group_{group_name}.csv")
    group_df.to_csv(filename, index=False)
    print(f"Saved: {filename}")

End up with 2 key datasets:   
Dataset 1: Separated datasets which are split by age groups. (5-year or yearly)  
Dataset 2: One complete dataset to contain all treatment history of each patient_id throughout all of their procedure time. (SVD)


## Data Segmentation

### Divide patient-level treatment dataset into three groups (V1, V2, V3) based on the continuity and duration of their visit history
Definitions:
- First Visit Date: Start of treatment history
- Last Visit Date: End of treatment history
- Procedure Data: The date which procedure occurred
- Max Gap: Largest time gap between consecutive visits
- Duration: Total time span = Last Visit Date - First Visit Date

V1 (Very Consistent): Max gap between visits is less than 2 years (Doesn’t matter how long the total history is, as long as it's consistent)  
V2 (Inconsistent but Long History): Max gap is 2 years or more; Total duration is 7 years or more  
V3 (Inconsistent and Short History): Everything else

In [8]:
def classify_patient(group):
    procedure_dates = group.sort_values('procedure_date')['procedure_date']
    
    # Visit span in years
    span_years = (group['last_visit'].iloc[0] - group['first_visit'].iloc[0]).days / 365.0

    # Maximum time gap between consecutive visits
    gaps = procedure_dates.diff().dropna().dt.days / 365.0
    max_gap = gaps.max() if not gaps.empty else 0

    # Classification
    if span_years >= 1 and max_gap <= 2:
        return 'V1'
    elif span_years >= 7:
        return 'V2'
    else:
        return 'V3'

# Apply the classification
patient_versions = df_next.groupby('patient_id').apply(classify_patient).reset_index()
patient_versions.columns = ['patient_id', 'version']

# Merge version info back to full dataset (Treatment Based)
df_next = df_next.merge(patient_versions, on='patient_id')

KeyError: 'procedure_date'

In [ ]:
df_v1 = df_next[df_next['version'] == 'V1']
df_v2 = df_next[df_next['version'] == 'V2']
df_v3 = df_next[df_next['version'] == 'V3']